In [31]:
from ultralytics import YOLO
from lib.lib_utils import Utils 
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
import cv2
import torch as Torch
import math

In [ ]:
#trasformazione .xyz to .png 
Utils.from_xyz_to_png(
    Path('/home/gabro/GrapheDetectProject/data.xyz/subset_xyz'), 
    Path('/home/gabro/GrapheDetectProject/dataset_imm_300'), 
    100)


In [ ]:
#divisione del dataset in train/ test/
#inserire percorso dataset da dividere e percentuale del test 
Utils.split_dataset('/home/gabro/GrapheDetectProject/data_yolo_new/', 0.2)   

In [32]:
# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("/home/gabro/GrapheDetectProject/best_100_campioni_new.pt")  # load a pretrained model (recommended for training)


In [ ]:
# Use the model
# model.train(data="coco128.yaml", epochs=3)  # train the model
model.train(data="config_yolov8.yaml", epochs=100)  # train the model

In [3]:
metrics = model.val()  # evaluate model performance on the validation set
results = model("/home/gabro/GrapheDetectProject/b74c9ce2-graphene_218641_bonds.png")  # predict on an image
success = model.export(format="onnx")  # export the model to ONNX format

Ultralytics YOLOv8.0.88 🚀 Python-3.11.3 torch-2.0.0+cu117 CPU
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


FileNotFoundError: '/content/gdrive/MyDrive/Materiale tirocinio Gabriele/GrapheDefectDetector/google_colab_config.yaml' does not exist

Inferenza e trasformazione boxes in immagini singole 

In [37]:
pathImmProva = "/home/gabro/GrapheDetectProject/b74c9ce2-graphene_218641_bonds.png"
results = model(pathImmProva)  # predict on an image

boxes = results[0].boxes
index = 0
print(boxes.data.size)
# for box in boxes.data:
#     print(box.xyxy)
for box in boxes:
    #devo trasformare xy (alto sx), xy (alto dx) in [start_row:end_row, start_col:end_col]
    print(boxes[index].xyxy)    #posizione angolo in alto a sx e in basso a dx in pixel
    array = Torch.Tensor.numpy(boxes[index].xyxy)
    # print(array.size)
    x1 = math.ceil(array[0,0])
    y1 = math.ceil(array[0,1])
    x2 = math.floor(array[0,2])
    y2 = math.floor(array[0,3])

    # res_plotted = results[0].plot(labels = False, line_width = 1)
    # plt.imshow(res_plotted)
    # plt.show()
    # cv2.imshow("result", res_plotted)

    #crop difetto
    img = cv2.imread(pathImmProva)
    print(img.shape) # Print image shape
    # cv2.imshow("original", img)
    
    # Cropping an image
    cropped_image = img[y1:y2, x1:x2] #img[start_row:end_row, start_col:end_col]

    # Display cropped image
    # cv2.imshow("cropped", cropped_image)

    nameCropped = pathImmProva.removesuffix('.png') + "_cropped_box_" + str(index)+ ".png"
    # Save the cropped image
    cv2.imwrite(nameCropped, cropped_image)
    
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    index = index+1




image 1/1 /home/gabro/GrapheDetectProject/b74c9ce2-graphene_218641_bonds.png: 640x640 2 defects, 298.8ms
Speed: 7.9ms preprocess, 298.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)


<built-in method size of Tensor object at 0x7fac63d53de0>
tensor([[ 4.3943, 18.4342, 34.3378, 50.7019]])


NameError: name 'Torch' is not defined